In [5]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

In [26]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [27]:
test_data.data.shape

torch.Size([10000, 28, 28])

In [8]:
'''
1. Torch tensor
train_target = torch.tensor(train['Target'].values.astype(np.float32)) # train ist ein DF
train = torch.tensor(train.drop('Target', axis = 1).values.astype(np.float32))

2. Torch dataset
train_tensor = data_utils.TensorDataset(train, train_target)
test_tensor = data_utils.TensorDataset(test, test_target)

3. Torch dataloader
train_loader = data_utils.DataLoader(dataset = train_tensor, batch_size = batch_size, shuffle = True)
test_loader = data_utils.DataLoader(dataset = test_tensor, batch_size = batch_size, shuffle = False)
'''


"\n1. Torch tensor\ntrain_target = torch.tensor(train['Target'].values.astype(np.float32)) # train ist ein DF\ntrain = torch.tensor(train.drop('Target', axis = 1).values.astype(np.float32))\n\n2. Torch dataset\ntrain_tensor = data_utils.TensorDataset(train, train_target)\ntest_tensor = data_utils.TensorDataset(test, test_target)\n\n3. Torch dataloader\ntrain_loader = data_utils.DataLoader(dataset = train_tensor, batch_size = batch_size, shuffle = True)\ntest_loader = data_utils.DataLoader(dataset = test_tensor, batch_size = batch_size, shuffle = False)\n"

In [9]:
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)


In [10]:
for X, y in train_dataloader:
    print(X.shape)
    break

torch.Size([64, 1, 28, 28])


In [23]:
# torch.full(size=(9,), fill_value=1)

In [12]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.Softmax()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

Using cpu device


In [13]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [14]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss() # mean absolute error , mse

In [15]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [16]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad() # fit
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [17]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [24]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.605838  [    0/60000]
loss: 1.622529  [ 6400/60000]
loss: 1.569944  [12800/60000]
loss: 1.647292  [19200/60000]
loss: 1.676535  [25600/60000]
loss: 1.608474  [32000/60000]
loss: 1.621572  [38400/60000]
loss: 1.673513  [44800/60000]
loss: 1.679422  [51200/60000]
loss: 1.678599  [57600/60000]
Test Error: 
 Accuracy: 78.4%, Avg loss: 0.026309 

Epoch 2
-------------------------------
loss: 1.617594  [    0/60000]
loss: 1.622148  [ 6400/60000]
loss: 1.589558  [12800/60000]
loss: 1.700701  [19200/60000]
loss: 1.628790  [25600/60000]
loss: 1.630915  [32000/60000]
loss: 1.602335  [38400/60000]
loss: 1.618191  [44800/60000]
loss: 1.695023  [51200/60000]
loss: 1.625638  [57600/60000]
Test Error: 
 Accuracy: 79.7%, Avg loss: 0.026110 

Epoch 3
-------------------------------
loss: 1.609054  [    0/60000]
loss: 1.572074  [ 6400/60000]
loss: 1.569366  [12800/60000]
loss: 1.630310  [19200/60000]
loss: 1.639643  [25600/60000]
loss: 1.649189  [32000/600

In [19]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [20]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [21]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"


In [22]:
test_data.data.shape

torch.Size([10000, 28, 28])